### Initialization

In [29]:
from dash import Dash, html, Output, Input, callback, State
from dash.exceptions import PreventUpdate
import dash.dcc as dcc
import dash_bootstrap_components as dbc
import ta

import yfinance as yf
import os
import pandas as pd
import numpy as np

In [4]:
def grab_ticker_data(symbol, time):
    DATA_PATH = f"../../data/{symbol}_data_{time}.json"
    if os.path.exists(DATA_PATH):
        # Read from file if we've already downloaded the data.
        with open(DATA_PATH) as f:
            ticker_hist = pd.read_json(DATA_PATH)

    else:
        ticker = yf.Ticker(symbol)
        ticker_hist = ticker.history(period=time)

        # Save file to json in case we need it later.  This prevents us from having to re-download it every time.
        ticker_hist.to_json(DATA_PATH)
    return ticker_hist

In [5]:
def create_shifted_data(ticker_hist, days=1):
    # Ensure we know the actual closing price
    data = ticker_hist.copy()
    del data["Stock Splits"]
    del data["Dividends"]
    # data = data.rename(columns = {'Close':'Actual_Close'})
    
    # Setup our target.  This identifies if the price went up or down
    # data["Target"] = ticker_hist.rolling(days+1).apply(lambda x: x.iloc[-1] > x.iloc[0])["Close"] # Might have to adjust if days are different
    # data["Target"] = (ticker_hist["Close"].shift(days) < ticker_hist["Close"]).astype(float)
    data["Target"] = (data["Close"].shift(-days) > data["Close"]).astype(int)

    # Shift stock prices forward one day, so we're predicting tomorrow's stock prices from today's prices.
    # ticker_prev = ticker_hist.copy()
    # ticker_prev = ticker_prev.shift(days)

    # data = data.join(ticker_prev[predictors]).iloc[1:]
    return data

### TA Generator | Dash Preview

In [6]:
symbol = "MSFT"
time = "max"
ticker_hist = grab_ticker_data(symbol, time)
data = create_shifted_data(ticker_hist)

In [7]:
submodules = [name for name in dir(ta) if not (name.startswith("__") or name.startswith("add") or name == 'wrapper' or name == 'utils')]

In [10]:
mod_to_inds = {}
    
for module in submodules:
    function_names = [
        name for name in dir(getattr(ta, module)) 
        if callable(getattr(getattr(ta, module), name)) and 
        not (name.startswith("__") or name.startswith("_") or name[0].isupper())]
    mod_to_inds[module] = function_names

In [12]:
import inspect

def get_function_arguments(func):
    signature = inspect.signature(func)
    arguments = []
    for name, param in signature.parameters.items():
        if param.default is inspect.Parameter.empty:
            arguments.append((name, None))
        else:
            arguments.append((name, param.default))
    return arguments

In [14]:
def draw_ind_inputs(module, ind):
    inputs = []
    for arg, val in get_function_arguments(getattr(getattr(ta, module), ind)):
        inputs.append(dbc.Stack([
            dbc.Label(arg),
            dbc.Input(id={'type':'ind_param', 'index':arg}, value=val)
        ], direction='horizontal', gap=2))
    return inputs


In [28]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
app.layout = dbc.Container([
    dcc.Store(id='ind-store', storage_type='memory'),
    dbc.Row(
        [
            dbc.Col(
                [
                    dbc.Stack([
                        dbc.Label("Indicator Type", style={'width': '50%'}),
                        dcc.Dropdown(submodules, id="submod-dd", style={'width':'100%'}),
                    ], direction='horizontal', gap=2)
                ]
            ),
            dbc.Col(
                [
                    dbc.Stack([
                        dbc.Label("Indicator"),
                        dcc.Dropdown(id = 'ind-dd', style={'width':'100%'})
                    ], direction='horizontal', gap=2)
                ]
            )
        ]
    ),
    html.Br(),
    dbc.Row(
        [
            dbc.Col((
                [
                    html.Div(id='ind-params'),
                    dbc.Button("Add Indicator", id='add-ind', color='primary')
                ]
            ))
        ]
    )
])

@app.callback(
    Output('ind-dd', 'options'),
    Input('submod-dd', 'value')
)
def update_ind_dropdown(submod):
    if not submod:
        raise PreventUpdate
    return mod_to_inds[submod]

@app.callback(
    Output('ind-params', 'children'),
    Input('ind-dd', 'value'),
    Input('submod-dd', 'value')
)
def update_ind_adder(ind, mod):
    if not ind or not mod:
        return []
    return draw_ind_inputs(mod, ind)

@app.callback(
    Output('ind-store', 'data'),
    State('ind-store', 'data'),
    Input('add-ind', 'n_clicks')
)
def add_indicator(data, clicks):
    if not clicks:
        raise PreventUpdate

    if not data:
        data = {}

    
    
    return data

app.run(jupyter_mode="inline")